In [41]:
! pip install lxml bs4 requests

In [42]:
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import lxml
import os

# Lấy thông tin từ portal.ptit.edu.vn

In [43]:
global notificationDataFile
global preprocessedData
global contentData

# Lấy đường dẫn thư mục hiện tại của file script
current_dir = os.getcwd()
current_dir = os.path.dirname(current_dir)
# Tạo thư mục result nếu chưa tồn tại
result_path = os.path.join(current_dir, 'result')
if not os.path.exists(result_path):
    os.makedirs(result_path)

# notificationDataFile = os.path.join(result_path, 'noti_from_hv.json')
# preprocessedData = os.path.join(result_path, 'data_preproc_from_hv.json') # data preprocessing file
# contentData = os.path.join(result_path, 'contens_from_hv.json')

# notificationDataFile = os.path.join(result_path, 'noti_from_pgv.json')
# preprocessedData = os.path.join(result_path, 'data_preproc_from_pgv.json') # data preprocessing file
# contentData = os.path.join(result_path, 'contens_from_pgv.json')

# notificationDataFile = os.path.join(result_path, 'news.json')
# preprocessedData = os.path.join(result_path, 'data_preproc_news.json') # data preprocessing file
# contentData = os.path.join(result_path, 'contens_news.json')

notificationDataFile = os.path.join(result_path, 'notifications.json')
preprocessedData = os.path.join(result_path, 'data_preproc_notifications.json') # data preprocessing file
contentData = os.path.join(result_path, 'contens_notifications.json')

# 40 with news and 5 with notifications (info from ~2023)
maxPage = 5 # Max is 14~15 with notifications from ptit and 15 with notifications from GV (info from ~2023)

# base_url = 'https://portal.ptit.edu.vn/giaovu/category/thong-bao/thong-bao-tu-ho%cc%a3c-vie%cc%a3n/'
# base_url = 'https://portal.ptit.edu.vn/giaovu/category/thong-bao/thong-bao-tu-phonggv/'
# base_url = 'https://portal.ptit.edu.vn/category/tin-tuc/'
base_url = 'https://portal.ptit.edu.vn/category/thong-bao/'

In [44]:
portalData = []

def scrapePortalData(url):
    global portalData
    response = requests.get(url)

    soup = BeautifulSoup(response.text, from_encoding='utf-8', features='lxml')
    divs = soup.find('div', class_='posts_group lm_wrapper classic col-3')
    info_s = divs.find_all('div')
    
    for item in info_s:
        info_item = {}
        # Tìm tiêu đề từ class "entry-title"
        title_tag = item.find('h2', class_='entry-title')
        info_item['Title'] = title_tag.get_text().strip() if title_tag else ''
        info_item['Link'] = title_tag.find('a')['href'] if title_tag and title_tag.find('a') else ''

        # Tìm nội dung từ class "post-excerpt"
        excerpt_tag = item.find('div', class_='post-excerpt')
        info_item['Content'] = excerpt_tag.get_text().strip() if excerpt_tag else ''

        # Tìm ngày đăng từ class "date_label"
        date_tag = item.find('div', class_='date_label')
        info_item['Date'] = date_tag.get_text().strip() if date_tag else ''

        portalData.append(info_item)

def saveDataToJson():
    with open(notificationDataFile, 'w', encoding='utf-8-sig') as file:
        json.dump(portalData, file, ensure_ascii=False, indent=4)

for i in range(1, maxPage + 1):
    scrapePortalData(base_url + 'page/' + str(i))

saveDataToJson()

In [45]:
# Xử lý dữ liệu JSON
def non_empty_fields(item):
    # Đếm số lượng trường không rỗng trong một item
    non_empty_count = sum(1 for value in item.values() if value)
    return non_empty_count, item

def convert_date(date_str):
    # Chuyển đổi ngày từ chuỗi sang đối tượng datetime, hoặc trả về None nếu không hợp lệ
    try:
        return datetime.strptime(date_str, '%d/%m/%Y')
    except (ValueError, TypeError):
        return None

def process_json_data(data):
    # Tạo list mới chỉ chứa items không hoàn toàn rỗng và thêm 'info_fullness' và 'Date' chuyển đổi
    non_empty_items = [item for item in data if any(item.values())]
    for item in non_empty_items:
        item['info_fullness'] = non_empty_fields(item)[0]
        item['Date'] = convert_date(item['Date']) if item['Date'] else None

    # Sắp xếp list dựa trên 'info_fullness', sau đó theo 'Date' nếu có
    not_empty_sorted = sorted(non_empty_items, key=lambda x: (-x['info_fullness'], x['Date'] if x['Date'] else datetime.min))

    # Loại bỏ những items trùng lặp, giữ lại items với 'info_fullness' cao nhất
    seen = set()
    unique_data = []
    for item in not_empty_sorted:
        # Tạo tuple của 'Title', 'Link' để kiểm tra trùng lặp
        title_link = (item.get('Title'), item.get('Link'))
        if title_link not in seen:
            seen.add(title_link)
            unique_data.append(item)

    # Cuối cùng, sắp xếp kết quả dựa theo 'Date' giảm dần
    final_data = sorted(unique_data, key=lambda x: x['Date'] if x['Date'] else datetime.min, reverse=True)
    
    # Loại bỏ field 'info_fullness' và item nào không có 'Date' hợp lệ
    final_cleaned_data = [item for item in final_data if item['Date']]
    # Chuyển đổi 'Date' thành chuỗi
    for item in final_cleaned_data:
        if item['Date']:
            item['Date'] = item['Date'].strftime('%d/%m/%Y')

    for item in final_cleaned_data:
        del item['info_fullness']
    
    return final_cleaned_data

# Chuẩn bị dữ liệu từ file JSON
with open(notificationDataFile, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Xử lý dữ liệu
cleaned_data = process_json_data(data)

# Ghi dữ liệu đã được làm sạch vào file JSON mới
with open(preprocessedData, 'w', encoding='utf-8-sig') as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

In [46]:
def get_contents_from_page(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text, from_encoding='utf-8', features='lxml')

    div_contents = soup.find('div', class_='the_content_wrapper')
    contents = div_contents.find_all(['p', 'li'], recursive=True)
    
    content_collection = ''
    if contents:
        for content in contents:
            content_collection = ' '.join([content_collection, content.get_text().strip()])

    return content_collection

# Đọc dữ liệu từ file JSON
with open(preprocessedData, 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Lấy nội dung từ mỗi liên kết và thêm vào mỗi item
for item in data:
    contents_url = item['Link']
    new_content = get_contents_from_page(contents_url)
    item['Content'] = new_content

# Ghi dữ liệu làm sạch kèm nội dung vào file JSON mới
with open(contentData, 'w', encoding='utf-8-sig') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [47]:
# Xóa file JSON tạm thời
os.remove(notificationDataFile)
os.remove(preprocessedData)